In [84]:
%reset -f

In [85]:
#!/usr/bin/env python3
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import torch
from torch import nn
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy
from torcheval.metrics.functional import binary_accuracy
torch.manual_seed(18)
torch.cuda.is_available()
from DiscriminatorV3 import DiscriminatorV3, ConvBlock
from DiscriminatorV4 import DiscriminatorV4, ConvBlock
from FacesDataset import FacesDataset
import matplotlib.pyplot as plt

In [86]:
def epoch_system_out_string(epoch:int, train_loss:float, train_acc:float, val_loss:float, val_acc:float, test_acc:float)->str:
    return (f'Epoch: {epoch} -- train Loss: {round(train_loss, 4)} \t valid Loss: {round(val_loss, 4)} \t train acc.:{round(train_acc, 4)} \t val acc.:{round(val_acc, 4)} \t test acc.:{round(test_acc, 4)}')

In [87]:
@torch.no_grad()
def estimate_performance(model, data_loader, device='cuda'):
    loss_value = 0.0
    acc_value = 0.0
    loss_fn = nn.BCELoss()
    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    model.eval()
    for batch in tqdm(data_loader):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs.flatten().float(), targets.float())
        metric.update(outputs.flatten(), targets)
        loss_value += loss.item()
        acc_value += metric.compute().item()
    return loss_value/data_loader.__len__(), acc_value/data_loader.__len__()

In [ ]:
def train(model, optimizer, train_loader, val_loader, test_loader, epochs, loss_fn, device='cuda'):
    model.to(device)
    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)

    for epoch in (range(epochs)):
        # train the model on the training set
        train_loss = 0.0
        train_acc = 0.0
        model.train()
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs.flatten().float(), targets.float())
            loss.backward()
            optimizer.step()
            metric.update(outputs.flatten(), targets)
            train_loss += loss.item()
            train_acc += metric.compute().item()

        # estimate the performance of the model on the validation set and the test set
        train_loss, train_acc = train_loss / len(train_loader), train_acc / len(train_loader)
        val_loss, val_acc = estimate_performance(model, val_loader, device)
        test_loss, test_acc = estimate_performance(model, test_loader, device)

        print(epoch_system_out_string(epoch, train_loss, train_acc, val_loss, val_acc, test_acc))

In [88]:
def train(model, optimizer, train_loader, val_loader, test_loader, epochs, loss_fn, device='cuda'):
    model.to(device)
    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)

    # Initialize empty dataframes
    loss_df = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss'])
    acc_df = pd.DataFrame(columns=['epoch', 'train_acc', 'val_acc'])

    for epoch in (range(epochs)):
        # train the model on the training set
        train_loss = 0.0
        train_acc = 0.0
        model.train()
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs.flatten().float(), targets.float())
            loss.backward()
            optimizer.step()
            metric.update(outputs.flatten(), targets)
            train_loss += loss.item()
            train_acc += metric.compute().item()

        # estimate the performance of the model on the validation set and the test set
        train_loss, train_acc = train_loss / len(train_loader), train_acc / len(train_loader)
        val_loss, val_acc = estimate_performance(model, val_loader, device)
        test_loss, test_acc = estimate_performance(model, test_loader, device)
        
        # Add to dataframes
        epoch_data = {'epoch': epoch+1,
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'test_loss': test_loss}
        loss_df = pd.concat([loss_df, pd.DataFrame(epoch_data, index=[0])], ignore_index=True)
        
        epoch_data = {'epoch': epoch+1,
                    'train_acc': train_acc,
                    'val_acc': val_acc,
                    'test_acc': test_acc}
        acc_df = pd.concat([acc_df, pd.DataFrame(epoch_data, index=[0])], ignore_index=True)

        print(epoch_system_out_string(epoch, train_loss, train_acc, val_loss, val_acc, test_acc))

    # save dataframes to csv files
    # Save data to CSV files
    loss_df.to_csv('outputs/losses_V4.csv', index=False)
    acc_df.to_csv('outputs/accuracies_V4.csv', index=False)

In [89]:
batch_size = 8

In [90]:
training_set = FacesDataset('datasets/train.csv')
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)

valid_set = FacesDataset('datasets/valid.csv')
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=True)

test_set = FacesDataset('datasets/test.csv')
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [91]:
model = DiscriminatorV4()

In [92]:
model.forward(torch.randn(1, 3, 256, 256)), model.forward(torch.randn(1, 3, 256, 256)).size()

(tensor([[0.4534]], grad_fn=<SigmoidBackward0>), torch.Size([1, 1]))

In [ ]:
loss_fn = nn.BCELoss()

params = model.parameters()
learning_rate = 3e-4
optimizer = torch.optim.Adam(params, lr=learning_rate)

# train the model
num_epochs = 15
train(model=model, optimizer=optimizer, train_loader=train_loader, val_loader=valid_loader, test_loader=test_loader,loss_fn=loss_fn, epochs=num_epochs)

In [94]:
# model.save()